In [2]:
!pip install yfinance
!pip install yahoofinancials

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15191 sha256=96c554d43bcf020c5bcffb4d16456f58a1a418eb04a1b07b84789a3ee88b02db
  Stored in directory: /root/.cache/pip/wheels/4b/63/46/e7110bfee88685fe69e338d1b14d1748921862aa57b6705b60
Successfully built yahoofinancials


In [3]:
import numpy as np
import time as tm
import datetime 
import tensorflow as tf
import pandas as pd
# Data preparation
from sklearn.preprocessing import MinMaxScaler
from collections import deque

# AI
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# Graphics library
import matplotlib.pyplot as plt

import yfinance as yf
from yahoofinancials import YahooFinancials
import time
import datetime

In [4]:
#time to pull the data
start_date =datetime.datetime.today().strftime('%Y-%m-%d')
end_date = (datetime.datetime.today() - datetime.timedelta(days=1100)).strftime('%Y-%m-%d')

In [5]:
# calling API 
data=YahooFinancials('amzn').get_historical_price_data(end_date,start_date,"daily")
#into the dataframe
amzn_df = pd.DataFrame(data['AMZN']['prices'])
amzn_df =amzn_df.drop(['high','low','open','volume','date','adjclose'], axis=1)
amzn_df.rename(columns={'formatted_date':'date'},inplace=True)
amzn_df['date']=pd.to_datetime(amzn_df['date'])

In [6]:
# SETTINGS

# Window size or the sequence length, 7 (1 week)
N_STEPS = 7

# Lookup steps, 1 is the next day, 3 = after tomorrow
LOOKUP_STEPS = [1, 2, 3]

In [7]:
# Scale data for ML engine
scaler = MinMaxScaler()
amzn_df['scaled_close'] = scaler.fit_transform(np.expand_dims(amzn_df['close'].values, axis=1))


In [8]:
def PrepareData(days):
    df = amzn_df.copy()
    df['future'] = df['scaled_close'].shift(-days)
    last_sequence = np.array(df[['scaled_close']].tail(days))
    df.dropna(inplace=True)
    sequence_data = []
    sequences = deque(maxlen=N_STEPS)
    
    for entry, target in zip(df[['scaled_close'] + ['date']].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == N_STEPS:
            sequence_data.append([np.array(sequences), target])
            
    last_sequence = list([s[:len(['scaled_close'])] for s in sequences]) + list(last_sequence)
    last_sequence = np.array(last_sequence).astype(np.float32)

    # construct the X's and Y's
    X, Y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        Y.append(target)

   # convert to numpy arrays
    X = np.array(X)
    Y = np.array(Y)

    return df, last_sequence, X, Y

In [9]:
PrepareData(3) # 3 days


(         close       date  scaled_close    future
 0    89.410004 2019-11-07      0.054307  0.049343
 1    89.293999 2019-11-08      0.053178  0.037230
 2    88.582497 2019-11-11      0.046253  0.037955
 3    88.900002 2019-11-12      0.049343  0.030602
 4    87.655502 2019-11-13      0.037230  0.036948
 ..         ...        ...           ...       ...
 751  96.790001 2022-11-01      0.126139  0.069588
 752  92.120003 2022-11-02      0.080684  0.065208
 753  89.300003 2022-11-03      0.053236  0.059855
 754  90.980003 2022-11-04      0.069588  0.022479
 755  90.529999 2022-11-07      0.065208  0.124581
 
 [756 rows x 4 columns], array([[0.19057335],
        [0.18113203],
        [0.12613884],
        [0.0806843 ],
        [0.05323638],
        [0.06958834],
        [0.0652083 ],
        [0.05985503],
        [0.02247909],
        [0.12458146]], dtype=float32), array([[[0.05430704726905322, Timestamp('2019-11-07 00:00:00')],
         [0.05317793546975691, Timestamp('2019-11-08 00:00:0

In [10]:
def GetTrainedModel(x_train, y_train):
  model = Sequential()
  model.add(LSTM(60, return_sequences=True, input_shape=(N_STEPS, len(['scaled_close']))))
  model.add(Dropout(0.3))
  model.add(LSTM(120, return_sequences=False))
  model.add(Dropout(0.3))
  model.add(Dense(20))
  model.add(Dense(1))

  BATCH_SIZE = 8
  EPOCHS = 80

  model.compile(loss='mean_squared_error', optimizer='adam')

  model.fit(x_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            verbose=1)

  model.summary()

  return model

In [11]:
# GET PREDICTIONS
predictions = []

for step in LOOKUP_STEPS:
  df, last_sequence, x_train, y_train = PrepareData(step)
  x_train = x_train[:, :, :len(['scaled_close'])].astype(np.float32)

  model = GetTrainedModel(x_train, y_train)

  last_sequence = last_sequence[-N_STEPS:]
  last_sequence = np.expand_dims(last_sequence, axis=0)
  prediction = model.predict(last_sequence)
  predicted_price = scaler.inverse_transform(prediction)[0][0]

  predictions.append(round(float(predicted_price), 2))

Epoch 1/80
94/94 [==============================] - 4s 9ms/step - loss: 0.0300
Epoch 2/80
94/94 [==============================] - 1s 14ms/step - loss: 0.0069
Epoch 3/80
94/94 [==============================] - 1s 15ms/step - loss: 0.0070
Epoch 4/80
94/94 [==============================] - 2s 16ms/step - loss: 0.0059
Epoch 5/80
94/94 [==============================] - 2s 17ms/step - loss: 0.0057
Epoch 6/80
94/94 [==============================] - 1s 16ms/step - loss: 0.0058
Epoch 7/80
94/94 [==============================] - 2s 16ms/step - loss: 0.0049
Epoch 8/80
94/94 [==============================] - 2s 17ms/step - loss: 0.0047
Epoch 9/80
94/94 [==============================] - 2s 16ms/step - loss: 0.0044
Epoch 10/80
94/94 [==============================] - 2s 17ms/step - loss: 0.0045
Epoch 11/80
94/94 [==============================] - 2s 18ms/step - loss: 0.0041
Epoch 12/80
94/94 [==============================] - 2s 16ms/step - loss: 0.0037
Epoch 13/80
94/94 [===================

In [12]:
if bool(predictions) == True and len(predictions) > 0:
  predictions_list = [str(d)+'$' for d in predictions]
  predictions_str = ', '.join(predictions_list)
  message = f' prediction for upcoming 3 days ({predictions_str})'
  
  print(message)

 prediction for upcoming 3 days (94.66$, 93.81$, 96.4$)


In [13]:
from google.colab import files
today=start_date =datetime.datetime.today()
day1=(today+datetime.timedelta(days=1)).strftime('%Y-%m-%d')
day2=(today+datetime.timedelta(days=2)).strftime('%Y-%m-%d')
day3=(today+datetime.timedelta(days=3)).strftime('%Y-%m-%d')
days=[day1,day2,day3]
amzn_lstm=pd.DataFrame()
amzn_lstm['future days']=days
amzn_lstm['price']=predictions
amzn_lstm['ticker']='AMZN'

amzn_lstm.to_csv('amzn_lstm.csv') 
files.download('amzn_lstm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>